# Question:
You are given Pyrolysis data containing temperatures and produced syngas volume. The independent variable is the temperature, two different temperatures are provided.
The task is to compare the means of the two groups and use independent sample t-test to determine if there is a significant difference. The confidence level should be 95%. 
Include in your results the group statistics and the t–test results.

We start off by just including the functions that we developed in the tutorial notebook.